In [1]:
# Install all required packages for the RAG system
!pip install transformers torch sentence-transformers faiss-cpu flask pyngrok pyyaml accelerate bitsandbytes
print("All dependencies installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.7 MB/s eta 0:00:00
All dependencies installed successfully!


In [ ]:
import os
import torch
import yaml
import faiss
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Complete knowledge base with 20 Shoplite documents
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": "To create a Shoplite account, users visit the registration page and provide their email address, password, and basic profile information. Email verification is required within 24 hours of registration. Users can choose between buyer accounts (free) or seller accounts (requires business verification). For buyer accounts, only basic personal information is needed. Seller accounts require additional business documentation including tax identification number and business registration proof. Once verified, users can access their dashboard, view order history, and manage their profile settings. Account recovery options include email verification and security questions."
    },
    {
        "id": "doc2",
        "title": "Shoplite Shopping Cart Features",
        "content": "The Shoplite shopping cart allows users to add multiple items from different sellers, apply promotional codes, and save items for later purchase. Cart contents are preserved for logged-in users across sessions. Users can adjust quantities, remove items, and see real-time shipping estimates. The cart displays item availability, seller information, and combined shipping costs when applicable. Special features include the 'save for later' option, price drop notifications, and bundled product suggestions. For mobile users, the cart syncs across devices and supports one-touch reordering of previous purchases."
    },
    {
        "id": "doc3",
        "title": "Shoplite Product Search and Filtering",
        "content": "Shoplite's search functionality allows users to find products using keywords, categories, or specific attributes. Advanced filtering options include price range, brand, customer ratings, and seller location. The search algorithm uses natural language processing to understand user intent and provides relevant results even with misspellings. Users can sort results by relevance, price (low to high or high to low), newest arrivals, or customer ratings. The platform also supports image search where users can upload photos to find similar products. Search history is saved for logged-in users to facilitate repeat purchases."
    },
    {
        "id": "doc4",
        "title": "Shoplite Checkout Process",
        "content": "The Shoplite checkout process is designed to be completed in under three minutes. After reviewing items in their cart, users proceed to checkout where they can select or add shipping addresses, choose delivery options, and apply payment methods. Shoplite supports multiple payment methods including credit/debit cards, digital wallets, and Shoplite Pay (the platform's proprietary payment system). Before final confirmation, users see a complete breakdown of costs including items, shipping, taxes, and any applicable discounts. Order confirmation is sent via email and SMS, and users can track their order status in real-time through their account dashboard."
    },
    {
        "id": "doc5",
        "title": "Shoplite Order Tracking and Delivery",
        "content": "Once an order is placed, Shoplite provides comprehensive tracking capabilities. Users can view their order status through the 'My Orders' section, which shows real-time updates from order confirmation to shipment to delivery. Each order is assigned a unique tracking number that can be used on the Shoplite website or app. Delivery options include standard shipping (5-7 business days), expedited shipping (2-3 business days), and same-day delivery in select metropolitan areas. Users receive notifications at key stages: order processed, order shipped, out for delivery, and delivered. For orders with multiple items from different sellers, each item is tracked separately."
    },
    {
        "id": "doc6",
        "title": "Shoplite Return and Refund Policies",
        "content": "Shoplite offers a 30-day return window for most products. Items must be in original condition with tags attached and in original packaging. To initiate a return, users go to their order history, select the item, and request a return authorization. Once approved, users can print a return shipping label or schedule a pickup. Refunds are processed to the original payment method within 5-7 business days after the returned item is received. Certain items such as personalized products, perishables, and digital downloads are not eligible for return. Shoplite also offers a 'returnless refund' option for low-cost items where the refund is issued without requiring the item to be returned."
    },
    {
        "id": "doc7",
        "title": "Shoplite Product Reviews and Ratings",
        "content": "Shoplite's review system allows verified buyers to rate products on a scale of 1-5 stars and leave detailed feedback. Reviews must include at least 20 characters and can include photos and videos. The platform uses AI to detect fake reviews and ensures only verified purchasers can contribute. Products display an aggregate rating based on all verified reviews, with recent reviews weighted more heavily. Users can filter reviews by rating, helpfulness, and recency. Sellers can respond to reviews to address customer concerns. Shoplite also offers incentives for high-quality reviews that include detailed information and media."
    },
    {
        "id": "doc8",
        "title": "Shoplite Seller Account Setup and Management",
        "content": "Setting up a seller account on Shoplite requires business verification and typically takes 2-3 business days. Sellers must provide business registration documents, tax identification, and bank account information. Once approved, sellers can create product listings, set prices, manage inventory, and configure shipping options. The seller dashboard provides analytics on sales performance, customer traffic, and inventory levels. Sellers can create promotions, offer discounts, and participate in Shoplite-sponsored marketing events. Shoplite charges sellers a commission fee ranging from 5% to 15% depending on the product category, plus a small transaction fee for each sale."
    },
    {
        "id": "doc9",
        "title": "Shoplite Inventory Management for Sellers",
        "content": "Shoplite provides sellers with comprehensive inventory management tools. Sellers can upload products individually or in bulk through CSV files. The system automatically updates inventory levels as sales are made and can send alerts when stock is low. Sellers can set minimum stock thresholds to automatically reorder products. For products with variations (size, color, etc.), each variation can have its own inventory count. The system also supports seasonal inventory management, allowing sellers to activate or deactivate products based on seasonality. Integration with third-party inventory management systems is available through Shoplite's API."
    },
    {
        "id": "doc10",
        "title": "Shoplite Commission and Fee Structure",
        "content": "Shoplite operates on a commission-based model for sellers. The commission rate varies by product category, ranging from 5% for books and media to 15% for electronics and luxury goods. In addition to commission, sellers pay a transaction fee of 2.5% + $0.30 per sale. Sellers who subscribe to Shoplite's Professional plan ($39.99/month) receive reduced commission rates and additional features. Payment processing fees are waived for sellers using Shoplite Pay. Sellers receive payouts twice monthly, with a 7-day holding period for new sellers. International sellers may incur additional currency conversion fees. All fee structures are clearly outlined in the seller agreement, with no hidden charges."
    },
    {
        "id": "doc11",
        "title": "Shoplite Customer Support Procedures",
        "content": "Shoplite offers 24/7 customer support through multiple channels including live chat, email, and phone. For immediate assistance, the live chat feature connects users with support agents within 2 minutes during peak hours. Email inquiries typically receive responses within 24 hours. Phone support is available in multiple languages for international customers. The support system is tiered, with simple issues resolved by frontline agents and complex problems escalated to specialized teams. Shoplite also maintains an extensive knowledge base with FAQs, video tutorials, and troubleshooting guides. For seller support, dedicated account managers are available for Professional plan subscribers."
    },
    {
        "id": "doc12",
        "title": "Shoplite Mobile App Features",
        "content": "The Shoplite mobile app offers all the functionality of the website with additional mobile-specific features. Available for both iOS and Android, the app supports biometric login for enhanced security. Mobile-exclusive features include barcode scanning for price comparison, AR product visualization, and location-based notifications for nearby deals. The app's interface is optimized for one-handed use and includes voice search capabilities. Push notifications alert users to order status updates, price drops on wish-listed items, and personalized promotions. The app also supports offline browsing of previously viewed products and allows users to create shopping lists that sync across devices."
    },
    {
        "id": "doc13",
        "title": "Shoplite API Documentation for Developers",
        "content": "Shoplite provides a comprehensive RESTful API for developers to integrate with the platform. The API supports authentication through OAuth 2.0 and returns data in JSON format. Key endpoints include product search, order management, inventory updates, and payment processing. Rate limits are set at 1000 requests per hour for standard access, with higher limits available for enterprise partners. The API documentation includes detailed parameter descriptions, request/response examples, and error code explanations. SDKs are available for Python, Java, JavaScript, and PHP. Webhook support enables real-time notifications for order status changes and inventory updates. All API communications are secured with TLS encryption."
    },
    {
        "id": "doc14",
        "title": "Shoplite Security and Privacy Policies",
        "content": "Shoplite employs industry-standard security measures to protect user data. All sensitive information is encrypted using AES-256 encryption both in transit and at rest. The platform undergoes regular security audits and penetration testing. Shoplite is GDPR compliant and provides users with control over their personal data through privacy settings. Two-factor authentication is available for all accounts and required for seller accounts. The platform uses machine learning to detect fraudulent activity and unusual account behavior. Shoplite does not sell user data to third parties and only shares information as required for order fulfillment or when legally compelled. Detailed privacy policies outline data collection, usage, and retention practices."
    },
    {
        "id": "doc15",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": "Shoplite offers various types of promotional codes and discounts to enhance customer experience. Promotional codes can be percentage-based (e.g., 10% off), fixed amount (e.g., $5 off), or shipping-related (e.g., free shipping). Codes can be applied at checkout and may have restrictions such as minimum purchase amounts, category limitations, or expiration dates. Shoplite also runs seasonal promotions, flash sales, and personalized offers based on user browsing history. Sellers can create their own promotional codes for their products, which are stackable with Shoplite-wide promotions. The platform tracks promotional effectiveness and provides analytics to both Shoplite and sellers. Users can manage their saved promotional codes in their account dashboard."
    },
    {
        "id": "doc16",
        "title": "Shoplite Payment Methods and Security",
        "content": "Shoplite accepts multiple payment methods to accommodate diverse customer preferences. These include major credit cards (Visa, MasterCard, American Express, Discover), digital wallets (Apple Pay, Google Pay, PayPal), and Shoplite's proprietary payment system, Shoplite Pay. All payment transactions are secured with end-to-end encryption and processed through PCI DSS compliant systems. Shoplite Pay offers additional benefits such as one-click checkout, extended return window, and purchase protection. For high-value transactions, Shoplite may implement additional verification steps. The platform uses advanced fraud detection algorithms to analyze transaction patterns and prevent unauthorized purchases. Payment information is tokenized and never stored in its entirety on Shoplite's servers."
    },
    {
        "id": "doc17",
        "title": "Shoplite Seller Performance Metrics",
        "content": "Shoplite evaluates seller performance using several key metrics to ensure quality customer experience. These include order defect rate (ODR), which measures returns, negative feedback, and customer service contacts; on-time shipping rate; and customer satisfaction score. Sellers are expected to maintain an ODR below 1% and an on-time shipping rate above 95%. Performance metrics are updated daily and visible in the seller dashboard. Sellers who consistently exceed standards may receive benefits such as increased visibility in search results and eligibility for special promotions. Conversely, sellers who fall below standards may face restrictions or account suspension. Shoplite provides resources and best practices to help sellers improve their performance metrics."
    },
    {
        "id": "doc18",
        "title": "Shoplite International Shipping Options",
        "content": "Shoplite supports international shipping to over 100 countries through partnerships with major carriers. International customers can see estimated delivery times and costs before completing their purchase. The platform automatically calculates customs duties and taxes, which can be paid at checkout or upon delivery. For sellers, Shoplite provides tools to configure international shipping rates and restrictions. International orders include tracking capabilities and insurance coverage. Customers in select countries can also use Shoplite's consolidation service, which allows multiple purchases to be combined into a single shipment to reduce costs. The platform handles all necessary customs documentation and provides clear information about import restrictions for different countries."
    },
    {
        "id": "doc19",
        "title": "Shoplite Subscription Services",
        "content": "Shoplite offers two subscription services: Shoplite Plus for buyers and Shoplite Professional for sellers. Shoplite Plus ($79/year) includes benefits such as free expedited shipping on eligible items, early access to sales, exclusive discounts, and enhanced customer support. Members also receive 5% back in Shoplite credits on eligible purchases. Shoplite Professional ($39.99/month) provides sellers with reduced commission rates, advanced analytics, promotional tools, and a dedicated account manager. Both subscriptions include a 30-day free trial and can be canceled at any time. Subscribers receive monthly usage reports and personalized recommendations to maximize the value of their membership. The subscription management interface allows users to upgrade, downgrade, or pause their subscriptions as needed."
    },
    {
        "id": "doc20",
        "title": "Shoplite Dispute Resolution Process",
        "content": "Shoplite provides a structured dispute resolution process to handle conflicts between buyers and sellers. When a customer is dissatisfied with a purchase, they can first contact the seller directly through the platform's messaging system. If unresolved after 48 hours, the customer can escalate the issue to Shoplite's resolution team. The team reviews all communication, order details, and any supporting evidence provided by both parties. Most disputes are resolved within 3-5 business days. Possible outcomes include full or partial refund, return authorization, or order cancellation. Shoplite's decision is binding, and both parties agree to abide by the resolution process when using the platform. The system maintains records of all disputes to identify patterns and take appropriate action against repeat offenders."
    }
]

print(f"Knowledge base loaded with {len(KNOWLEDGE_BASE)} documents")

In [ ]:
# Convert YAML prompts to Python dictionaries
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant with comprehensive knowledge of the Shoplite e-commerce platform.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation. Be concise, helpful, and cite your sources.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible",
            "Do not speculate or make up information",
            "If information is not in the context, state that you don't have that information"
        ],
        "response_format": "Answer: [Your response based on context]\nSources: [List document titles referenced]"
    },
    "complex_question_prompt": {
        "role": "You are a knowledgeable Shoplite support specialist trained to handle complex questions that require information from multiple documents.",
        "goal": "Synthesize information from multiple sources to provide comprehensive answers to complex questions about Shoplite's features and policies.",
        "context_guidelines": [
            "Identify and combine relevant information from multiple documents",
            "Organize information in a logical structure",
            "Clearly indicate when information comes from different sources",
            "Address all parts of multi-part questions"
        ],
        "response_format": "Answer: [Comprehensive response addressing all aspects of the question]\nSources: [List all document titles referenced, organized by relevance]"
    },
    "no_context_prompt": {
        "role": "You are a responsible Shoplite customer service representative who prioritizes accuracy over speculation.",
        "goal": "Politely decline to answer questions when no relevant context is found in the provided documents.",
        "context_guidelines": [
            "Carefully review all provided context to ensure no relevant information exists",
            "Do not attempt to answer questions outside your knowledge base",
            "Suggest alternative ways the customer might find the information they need"
        ],
        "response_format": "Answer: [Polite explanation that you cannot answer based on available information]\nSuggestion: [Alternative approach for the customer to find the information]\nSources: [None]"
    },
    "clarification_prompt": {
        "role": "You are a thoughtful Shoplite assistant who seeks to fully understand customer needs before providing answers.",
        "goal": "Ask clarifying questions when the customer's query is ambiguous, vague, or could be interpreted in multiple ways.",
        "context_guidelines": [
            "Identify specific aspects of the question that are unclear",
            "Formulate targeted questions to resolve ambiguity",
            "Explain why clarification is needed",
            "Offer possible interpretations if appropriate"
        ],
        "response_format": "Clarification needed: [Explanation of what's unclear]\nQuestions: [List of clarifying questions]\nSources: [None until clarification is provided]"
    }
}

print("Prompts loaded successfully")

In [ ]:
# Load the sentence transformer model for embeddings
print("Loading sentence transformer model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_model = embedding_model.to(device)

# Load the LLM with quantization for GPU constraints
print("Loading LLM...")
# Using an open model that doesn't require special access
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Alternative open model

# Configure quantization
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("LLM loaded successfully with quantization")

In [ ]:
# Create document embeddings
def create_embeddings(documents):
    """Create embeddings for all documents in the knowledge base."""
    print("Creating embeddings for documents...")
    texts = [doc["content"] for doc in documents]
    embeddings = embedding_model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()

# Build FAISS index
def build_faiss_index(embeddings):
    """Build a FAISS index for efficient similarity search."""
    print("Building FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Retrieve relevant documents
def retrieve_documents(query, index, documents, k=3):
    """Retrieve the top-k most relevant documents for a query."""
    # Create embedding for the query
    query_embedding = embedding_model.encode([query], convert_to_tensor=True)
    query_embedding = query_embedding.cpu().numpy()

    # Search in FAISS index
    distances, indices = index.search(query_embedding, k)

    # Return the top-k documents
    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs

# Format prompt with context
def format_prompt(prompt_type, query, context_docs=None):
    """Format the prompt based on the prompt type and context."""
    prompt_config = PROMPTS[prompt_type]

    # Start with the role
    prompt = f"Role: {prompt_config['role']}\n\n"

    # Add the goal
    prompt += f"Goal: {prompt_config['goal']}\n\n"

    # Add context guidelines
    prompt += "Guidelines:\n"
    for guideline in prompt_config['context_guidelines']:
        prompt += f"- {guideline}\n"
    prompt += "\n"

    # Add context if available
    if context_docs:
        prompt += "Context:\n"
        for i, doc in enumerate(context_docs):
            prompt += f"Document {i+1} ({doc['title']}): {doc['content']}\n\n"

    # Add the query
    prompt += f"User Query: {query}\n\n"

    # Add response format
    prompt += f"Response Format:\n{prompt_config['response_format']}\n\n"

    # Add the response instruction
    prompt += "Response:"

    return prompt

# Generate response using LLM
def generate_response(prompt, max_length=1000):
    """Generate a response using the LLM."""
    # Format the messages for the model
    messages = [
        {"role": "system", "content": "You are a helpful assistant for Shoplite e-commerce platform."},
        {"role": "user", "content": prompt}
    ]

    # Apply the chat template
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate the response
    outputs = pipe(
        formatted_prompt,
        max_new_tokens=max_length,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    # Extract the generated text
    response = outputs[0]["generated_text"]

    # Remove the prompt from the response
    response = response.replace(formatted_prompt, "").strip()

    return response

# Complete RAG pipeline
def rag_pipeline(query, prompt_type="base_retrieval_prompt", k=3):
    """Complete RAG pipeline: retrieve documents and generate response."""
    # Retrieve relevant documents
    retrieved_docs = retrieve_documents(query, faiss_index, KNOWLEDGE_BASE, k)

    # Format prompt with context
    prompt = format_prompt(prompt_type, query, retrieved_docs)

    # Generate response
    response = generate_response(prompt)

    # Return the response and retrieved documents
    return {
        "response": response,
        "retrieved_documents": [{"id": doc["id"], "title": doc["title"]} for doc in retrieved_docs]
    }

# Initialize the RAG system
print("Initializing RAG system...")
document_embeddings = create_embeddings(KNOWLEDGE_BASE)
faiss_index = build_faiss_index(document_embeddings)
print("RAG system initialized successfully!")

In [ ]:
# Initialize Flask app
app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint."""
    return jsonify({"status": "healthy", "message": "Shoplite RAG API is running"})

@app.route('/ping', methods=['POST'])
def ping_llm():
    """Direct LLM interaction without RAG."""
    try:
        data = request.json
        query = data.get('query', '')

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Format prompt without context
        prompt = format_prompt("base_retrieval_prompt", query, context_docs=None)

        # Generate response
        response = generate_response(prompt)

        return jsonify({
            "query": query,
            "response": response,
            "retrieved_documents": []
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/chat', methods=['POST'])
def chat_rag():
    """RAG chat endpoint."""
    try:
        data = request.json
        query = data.get('query', '')
        prompt_type = data.get('prompt_type', 'base_retrieval_prompt')
        k = data.get('k', 3)

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Process through RAG pipeline
        result = rag_pipeline(query, prompt_type, k)

        return jsonify({
            "query": query,
            "response": result["response"],
            "retrieved_documents": result["retrieved_documents"]
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

print("Flask API setup complete")

In [ ]:
# Get ngrok token from user input
ngrok_token = input("Enter your ngrok token: ")

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Start Flask app in a background thread
from threading import Thread

def run_app():
    app.run(port=5000)

thread = Thread(target=run_app)
thread.daemon = True
thread.start()

# Create ngrok tunnel
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel available at: {public_url}")

In [ ]:
import requests
import json

def test_api():
    """Test the API endpoints."""
    base_url = public_url

    # Test health endpoint
    print("Testing health endpoint...")
    response = requests.get(f"{base_url}/health")
    print(f"Health check status: {response.status_code}")
    print(f"Health check response: {response.json()}")

    # Test ping endpoint
    print("\nTesting ping endpoint...")
    ping_data = {"query": "Hello, how are you?"}
    response = requests.post(f"{base_url}/ping", json=ping_data)
    print(f"Ping status: {response.status_code}")
    print(f"Ping response: {response.json()}")

    # Test chat endpoint
    print("\nTesting chat endpoint...")
    chat_data = {"query": "How do I create a seller account on Shoplite?"}
    response = requests.post(f"{base_url}/chat", json=chat_data)
    print(f"Chat status: {response.status_code}")
    print(f"Chat response: {response.json()}")

# Run the tests
test_api()

In [ ]:
# Additional test cases for the RAG system
def run_additional_tests():
    """Run additional test cases to verify the RAG system."""
    base_url = public_url

    test_questions = [
        "What are Shoplite's return policies?",
        "How does order tracking work?",
        "What payment methods does Shoplite accept?",
        "How do I set up a seller account?",
        "What are the benefits of Shoplite Plus?"
    ]

    print("\nRunning additional test cases...")

    for i, question in enumerate(test_questions, 1):
        print(f"\nTest {i}: {question}")
        chat_data = {"query": question}
        response = requests.post(f"{base_url}/chat", json=chat_data)

        if response.status_code == 200:
            result = response.json()
            print(f"Response: {result['response'][:200]}...")
            print(f"Sources: {[doc['title'] for doc in result['retrieved_documents']]}")
        else:
            print(f"Error: {response.status_code} - {response.text}")

# Run additional tests
run_additional_tests()

In [ ]:
# Save the ngrok URL to a file for the chat interface
with open('ngrok_url.txt', 'w') as f:
    f.write(public_url)

print(f"ngrok URL saved to ngrok_url.txt: {public_url}")
print("\nTo use the chat interface, run the following command on your local machine:")
print(f"python src/chat-interface.py --url {public_url}")